In [68]:
api_key = "" #change this to your personal google api key 

ch_id = "UCdUcjkyZtf-1WJyPPiETF1g" #Youtube channel id

https://youtu.be/D56_Cx36oGY 

이분 영상 참고해서 만들었습니당
위의 api_key는 꼭 개인 api key로 바꿔서 사용해주세요!

In [69]:
import googleapiclient.discovery
import googleapiclient.errors
import pandas as pd
import numpy as np

In [70]:
# youtube data api access

api_service_name = "youtube"
api_version = "v3"

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=api_key)

get_channel_info() 는 한 채널에 대한 기본적인 정보 모으는 함수!

In [71]:
def get_channel_info(youtube, channel_id):
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=channel_id
    )
    response = request.execute()
    all_data = []

    for item in response['items']:
        data = {'channelName': item['snippet']['title'],
        'subscribers': item['statistics']['subscriberCount'],
        'totalVideos': item['statistics']['videoCount'],
        'playlistId': item['contentDetails']['relatedPlaylists']['uploads']

        }
        all_data.append(data)

    return(pd.DataFrame(all_data))

여기서 playlist_id 는 채널의 모든 영상이 들어있는 재생목록 id!

In [72]:
channel_stats = get_channel_info(youtube, ch_id)
playlist_id = channel_stats['playlistId'][0]
channel_stats

,channelName,subscribers,totalVideos,playlistId
0,ITSub잇섭,2320000,1019,UUdUcjkyZtf-1WJyPPiETF1g


한 재생목록 id를 넣으면 그 재생목록 속 영상들의 video id들을 긁어모아주는 함수!

In [73]:
def get_video_id(youtube, playlist_id, page_token):

    request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId=playlist_id,
        maxResults = 50,
        pageToken = page_token
    )
    response = request.execute()
    video_ids = []

    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])
    
    # It will take a long time to get all 6427 video ids if you uncomment this part
    
    next_page_token = response.get('nextPageToken')


    return video_ids, next_page_token

영상 id를 넣으면 우리가 원하는 정보를 뱉어주는 함수!

In [79]:
def get_video_info(youtube, video_ids):

    request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=video_ids
        )
    response = request.execute()

    video_info = []

    for video in response['items']:
        keep = {
            'Published': video['snippet']['publishedAt'],
            'HDThumbnailUrl': video['snippet']['thumbnails']['high']['url'],
            'ViewCount': video['statistics']['viewCount'],
            'VideoId': video['id'],
            'Title': video['snippet']['title'],
            #'channelid': video['snippet']['channelId']
            }

        video_info.append(keep)

    return(pd.DataFrame(video_info))
    
    

In [75]:
def get_video_id_1st(youtube, playlist_id):

    request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId=playlist_id,
        maxResults = 50
    )
    response = request.execute()
    video_ids = []

    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])
    
      
    next_page_token = response.get('nextPageToken')


    return video_ids, next_page_token

In [76]:
def get_pl_count(youtube, playlist_ids):

    request = youtube.playlists().list(
        part = "contentDetails",
        id = playlist_ids        
        )
    response = request.execute()

    pl_count = response['items'][0]['contentDetails']['itemCount']
    
    return pl_count


In [77]:
#playlist_id = "UUdUcjkyZtf-1WJyPPiETF1g"
num = get_pl_count(youtube, playlist_id)

r = int(np.ceil(num/50))
id_lst = []

video_ids, next_token = get_video_id_1st(youtube, playlist_id)

id_lst.append(video_ids)

for i in range(r-1):
    video_ids, next_token = get_video_id(youtube, playlist_id, next_token)
    id_lst.append(video_ids)

In [80]:
pages = []
for page in id_lst:
    video_df = get_video_info(youtube, page)
    pages.append(video_df)

creator = "ITSub잇섭"
df = pd.concat(pages, ignore_index=True)
df.to_csv(f"{creator}.csv")